# DT Assignment1

# Data Loading

In [123]:
import pandas as pd 
import numpy as np

In [124]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

### Gini(Di) = $"1-\sum(p^2)"$

In [185]:
def get_gini(df, label):
    n = len(df)
    if n != 0:
        gini = 1-(len(df.loc[df[label]=='yes']) / n)**2 - (len(df.loc[df[label]=='no']) / n)**2
    else:
        gini = 1
    return gini    

* 위의 확률식을 그대로 적용하기 위해 label(class_buys_computer)이 yes,no일때를 따로 추출하여 gini함수를 만들었다. n이 0일때는 ZeroDivisionError가 나타나기 때문에 n이 0이면 gini가 1로 수렴하게끔 작성해주었다.

* 밑에는 오류를 방지하는 다른 방법들이다.(멘토링에서 도움받았다)

In [202]:
from functools import reduce

In [229]:
#not_new라는 df는 get_attribute_gini_index구현할때 쓰이는 df인데 아무것도 안담겨 있어서 not_new를 가지고 해보았다.
reduce(lambda x, y : x/(y / n), not_new['class_buys_computer'].value_counts(),1)

1

In [230]:
#상대적인 값을 사용하는 nomarlize를 통해서도 오류를 없앨 수 있다.
1 - (not_new['class_buys_computer'].value_counts(normalize = True)**2).sum()

1.0

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [127]:
import itertools
def get_binary_split(df, attribute) : 
    unique = df[attribute].unique() #attribute의 unique한값만 뽑아서 저장한다.
    
    a = list(itertools.combinations(unique,1)) # itertools의 combinations을 이용하여 1개짜리를 만든다.
    b = list(itertools.combinations(unique,2)) # itertools의 combinations을 이용하여 2개짜리를 만든다.

    
    result = a+b
    return result

In [128]:
get_binary_split(pd_data,"age")

[('youth',),
 ('middle_aged',),
 ('senior',),
 ('youth', 'middle_aged'),
 ('youth', 'senior'),
 ('middle_aged', 'senior')]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

* 멘토링에서 많은 도움을 받았다

In [186]:
def get_attribute_gini_index(df, attribute, label):
    a = get_binary_split(df, attribute) #gini는 이진분류를 전제로 분석하기때문에 위에서 정의한 이진분류함수를 이용하여 가져온다.
    result={} #결과가 dictionary 형태여서 빈 dict를 만들어준다
    len_df = len(df)
    new = pd.DataFrame()
    
    for i in a :
#         for j in i: 
#             print(i, j)
#             b= df[df[attribute]==j]
            
#             new = pd.concat([new, b])
        target_idx = df[attribute].map(lambda x: x in i) #위에있는 for문을 간단하게 만든 방법이다.
        #이진분류를 받은 a에서 해당 인덱스를 target_idx에 저장한다.
        new = df[target_idx] #target_idx를 새로운 df인 new에 저장한다.
        b = new.shape[0] #new의 개수를 세어 b에 저장한다.
        not_new = df[~target_idx] #해당하는 데이터가 아닌 다른데이터를 not_new에 저장한다.
       
        gini = (b/len_df)*get_gini(new, label)+ ((len_df-b)/len_df)*get_gini(not_new, label)
                #해당하는 gini                    해당하지 않는 나머지 gini
        result[i] = gini
    return result

* 위의 코드를 이해하기위해 따로 실행해본 것들을 첨부하였다.

In [212]:
pd_data[pd_data.age.map(lambda x: x in ('youth', 'middle_aged'))]

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
10,youth,medium,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes


In [213]:
pd_data[(pd_data["age"] == "youth")]

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
10,youth,medium,yes,excellent,yes


In [214]:
pd_data[~(pd_data["age"] == "youth")]

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
9,senior,medium,yes,fair,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes
13,senior,medium,no,excellent,no


In [215]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{('youth',): 0.3936507936507937,
 ('middle_aged',): 0.35714285714285715,
 ('senior',): 0.4571428571428572,
 ('youth', 'middle_aged'): 0.4571428571428572,
 ('youth', 'senior'): 0.35714285714285715,
 ('middle_aged', 'senior'): 0.3936507936507937}

In [74]:
#정답
#get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth': 0.3936507936507936,
 'middle_aged': 0.35714285714285715,
 'youth_middle_aged': 0.4571428571428572,
 'senior': 0.45714285714285713,
 'youth_senior': 0.35714285714285715,
 'middle_aged_senior': 0.39365079365079364}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

In [216]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

(('middle_aged',), 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [132]:
##문제1 답안
get_binary_split(pd_data, "income")

[('high',),
 ('medium',),
 ('low',),
 ('high', 'medium'),
 ('high', 'low'),
 ('medium', 'low')]

In [204]:
##문제2 답안
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{('youth',): 0.3936507936507937,
 ('middle_aged',): 0.35714285714285715,
 ('senior',): 0.4571428571428572,
 ('youth', 'middle_aged'): 0.4571428571428572,
 ('youth', 'senior'): 0.35714285714285715,
 ('middle_aged', 'senior'): 0.3936507936507937}

* 모든 입력값들의 gini_index와 그중에서도 가장 작은 값을 각각 실행시켜 구했습니다.

In [208]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

(('middle_aged',), 0.35714285714285715)

In [205]:
get_attribute_gini_index(pd_data, "income", "class_buys_computer")

{('high',): 0.4428571428571429,
 ('medium',): 0.4583333333333333,
 ('low',): 0.45,
 ('high', 'medium'): 0.45,
 ('high', 'low'): 0.4583333333333333,
 ('medium', 'low'): 0.4428571428571429}

In [209]:
min(get_attribute_gini_index(pd_data, "income", "class_buys_computer").items())

(('high',), 0.4428571428571429)

In [206]:
get_attribute_gini_index(pd_data, "student", "class_buys_computer")

{('no',): 0.3673469387755103,
 ('yes',): 0.3673469387755103,
 ('no', 'yes'): 0.4591836734693877}

In [210]:
min(get_attribute_gini_index(pd_data, "student", "class_buys_computer").items())

(('no',), 0.3673469387755103)

In [207]:
get_attribute_gini_index(pd_data, "credit_rating", "class_buys_computer")

{('fair',): 0.42857142857142855,
 ('excellent',): 0.42857142857142855,
 ('fair', 'excellent'): 0.4591836734693877}

In [211]:
min(get_attribute_gini_index(pd_data, "credit_rating", "class_buys_computer").items())

(('excellent',), 0.42857142857142855)

* (('middle_aged',), 0.35714285714285715)으로 middle_aged가 있는 age라는 변수가 gini_index값이 가장작기때문에 가장 중요한 변수라고 할 수 있다.

In [220]:
##문제3 답안
#age가 가장 중요변수로 뽑혔고 그중에서도 middle_aged가 중요변수이다. 
#일단 youth,middle_aged,senior로 각각 데이터를 뽑은다음 youth와 senior을 합쳐 새로운 df를 생성하여 그 안에서 그다음으로 중요한 변수를 뽑을것이다. 
youth = pd_data.loc[pd_data['age']=='youth']
middle_aged = pd_data.loc[pd_data['age']=='middle_aged'] 
senior = pd_data.loc[pd_data['age']=='senior']
df = pd.concat([youth,senior])

In [219]:
df

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
10,youth,medium,yes,excellent,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
9,senior,medium,yes,fair,yes
13,senior,medium,no,excellent,no


* middle_aged가 제거된 새로운 df가 생성되었음을 확인할 수 있다. 이제 문제2번에서 했던것과 마찬가지로 중요변수를 살펴볼것이다.

In [222]:
get_attribute_gini_index(df, "income", "class_buys_computer")

{('high',): 0.375,
 ('medium',): 0.48,
 ('low',): 0.47619047619047616,
 ('high', 'medium'): 0.47619047619047616,
 ('high', 'low'): 0.48,
 ('medium', 'low'): 0.375}

In [223]:
min(get_attribute_gini_index(df, "income", "class_buys_computer").items())

(('high',), 0.375)

In [224]:
get_attribute_gini_index(df, "student", "class_buys_computer")

{('no',): 0.31999999999999984,
 ('yes',): 0.31999999999999984,
 ('no', 'yes'): 0.5}

In [225]:
min(get_attribute_gini_index(df, "student", "class_buys_computer").items())

(('no',), 0.31999999999999984)

In [226]:
get_attribute_gini_index(df, "credit_rating", "class_buys_computer")

{('fair',): 0.4166666666666667,
 ('excellent',): 0.4166666666666667,
 ('fair', 'excellent'): 0.5}

In [227]:
min(get_attribute_gini_index(df, "credit_rating", "class_buys_computer").items())

(('excellent',), 0.4166666666666667)

* student가 age다음으로 중요변수로 선정되었고 그중에서도 (('no',), 0.31999999999999984)가 가장작게 나왔다.